<a href="https://colab.research.google.com/github/BrystofKlazek/RAD/blob/main/code/01RAD_Ex12_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle house data set



## Downloading the Kaggle house rent dataset

The dataset we will use comes from Kaggle:

- *House Rent Prediction Dataset*  
  https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset/data

To download directly from Kaggle inside this notebook you need a Kaggle
API token (see *Account ? API ? Create New Token* on Kaggle). The cell
below assumes you have configured your `KAGGLE_USERNAME` and
`KAGGLE_KEY` environment variables or placed `kaggle.json` in the
standard location.

Exampe of Auto NB - let's beat it

https://www.kaggle.com/code/sahityasetu/boosting-algorithms-for-machine-learning


In [ ]:

# Download the Kaggle house rent dataset using kagglehub (no API key needed for public data)
try:
    import kagglehub  # lightweight helper for Kaggle datasets
except ImportError:  # pragma: no cover
    %pip install -q kagglehub
    import kagglehub

# Download latest version of the dataset; this returns a local directory path
path = kagglehub.dataset_download("iamsouravbanerjee/house-rent-prediction-dataset")
print("Path to dataset files:", path)


In [ ]:

from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from patsy import dmatrices


# `path` is a directory returned by kagglehub; locate the CSV inside it
dataset_dir = Path(path)
candidates = list(dataset_dir.rglob("House_Rent_Dataset.csv"))
if not candidates:
    raise FileNotFoundError(f"House_Rent_Dataset.csv not found under {dataset_dir}")

csv_path = candidates[0]
print("Loading data from:", csv_path)
house = pd.read_csv(csv_path)
print("Shape:", house.shape)
house.head()



## Questions for a linear regression analysis of house rent

When building a linear regression model for rent, it is useful to think
in terms of a workflow:

1. **Understand the data**
   - What is the response variable (e.g. `Rent`)?  
     What are the main predictor types (numeric, categorical, locations,
     amenities)?
   - Are there obvious data quality issues (missing values, impossible
     values, outliers)?

2. **Preprocessing and feature engineering**
   - How should categorical variables (e.g. city, furnishing status,
     point of contact) be encoded for a linear model (one?hot encoding,
     target encoding, etc.)?
   - Which numeric variables might benefit from scaling (standardization
     or robust scaling), and why can this matter for regularized
     regression?
   - Are there interactions that are conceptually meaningful
     (e.g. `BHK , Size`, `City , DistanceFromMainArea`)?
   - Can we create more interpretable features (e.g. rent per square
     foot, distance to city centre bins)?

3. **Transformations of response and regressors**
   - Is the distribution of `Rent` highly skewed or heavy tailed? Would a
     log transformation (modeling $\log(\text{Rent})$) stabilize
     variance and make residuals closer to normal?
   - Do some predictors show non linear relationships with rent? Would
     polynomial terms, splines, or monotone transforms (log, square
     root) be appropriate?
   - Are there predictors that should be centered or standardized before
     creating interaction or polynomial terms?

4. **Model specification and selection**
   - Start with a simple baseline: which variables should be included in
     a first OLS model, and how do residual plots look?
   - How to compare alternative specifications
     (different sets of features, transformed vs untransformed variables)
     using cross validation or a validation set?
   - When is it useful to move from plain OLS to regularized models such
     as ridge or lasso (e.g. many correlated predictors, high variance)?

5. **Model evaluation and diagnostics**
   - How to check linear model assumptions: residual vs fitted plots,
     QQ plots, heteroscedasticity, influential observations?
   - Which error metrics are most relevant here
     (RMSE, MAE, MAPE)?  How do training and test errors compare
     (overfitting vs underfitting)?
   - Are there systematic groups of houses (by city, BHK, furnishing)
     for which the model performs much worse, suggesting missing
     structure or interactions?



### More detailed questions to explore

- **Preprocessing**
  - How should missing values be handled for each variable (impute,
    drop, or create explicit missing indicators)?
  - Do we need to cap or Winsorize extreme values of `Rent` or `Size`
    before fitting a linear model?
  - Are there rare categories (e.g. cities or furnishing statuses with
    very few observations) that should be grouped together?

- **Transformations and linearity**
  - Plot `Rent` (or $\log(\text{Rent})$) against key predictors:
    `Size`, `BHK`, `Bathroom`, `City`, etc.  Do the relationships look
    approximately linear after transformation?
  - Would modeling $\log(\text{Rent})$ make residuals more symmetric and
    reduce heteroscedasticity?

- **Multicollinearity and regularization**
  - Are some predictors strongly correlated (e.g. `Size` and `BHK`)?  How
    do VIFs and condition numbers look for the chosen design matrix?
  - How do ridge and lasso behave in this dataset in terms of coefficient
    shrinkage and variable selection?
  - Which predictors consistently get selected by lasso across
    cross?validation folds?

- **Model selection and validation**
  - How does test error change when we:
    1. Add more predictors,
    2. Add interaction terms,
    3. Add polynomial terms,
    4. Switch from OLS to ridge/lasso?
  - How to choose the final model: by minimum cross?validated RMSE,
    parsimony (fewest predictors), or domain interpretability?

Use these questions as a checklist to design your own modeling pipeline
for the house rent dataset using linear regression and its regularized
variants.


In [ ]:
df = house.copy()

df.shape
df.isna().sum()
df.duplicated().sum()
df.dtypes


All shapes and Nans are fine

In [ ]:
print(df.describe(percentiles=[.01,.05,.1,.25,.5,.75,.9,.95,.99]))
print(df["Rent"].max(), df["Size"].min(), df["Size"].max())

df["log_rent"] = np.log(df["Rent"])


In [ ]:
for col in ['Floor', 'Area Type', 'Area Locality', 'City', 'Furnishing Status', 'Tenant Preferred']:
    print(f"Unique values for '{col}':")
    print(df[col].unique())
    print("\n")
    print(f"Column '{col}': {df[col].nunique()} unique values")

Logrent seems reasonable

See how many values there are to create either one-hot encoding ( Categories - C in the statmodels) or to merge them and then treat them as Categories

In [ ]:
import re

def parse_floor(floor_str):
    floor_str = str(floor_str).lower().strip()
    current_floor = None
    total_floors = None

    # Handle 'X out of Y' format
    match_out_of = re.match(r'(.+) out of (\d+)', floor_str)
    if match_out_of:
        current_part = match_out_of.group(1).strip()
        total_floors = int(match_out_of.group(2))
    else:
        current_part = floor_str

    if current_part == 'ground':
        current_floor = 0
    elif current_part == 'upper basement':
        current_floor = -1
    elif current_part == 'lower basement':
        current_floor = -2
    else:
        try:
            current_floor = int(current_part)
        except ValueError:
            pass

    # If total_floors wasn't extracted from 'out of Y' and current_floor is known and non-negative,
    # assume total_floors is the same as current_floor (e.g., "3" means 3 out of 3).
    if total_floors is None and current_floor is not None and current_floor >= 0:
        total_floors = current_floor

    if current_floor is None or total_floors is None:
        raise ValueError(f"Could not parse floor string: '{floor_str}'")

    return current_floor, total_floors

df[['current_floor', 'total_floors']] = df['Floor'].apply(lambda x: pd.Series(parse_floor(x)))

print("Original 'Floor' column unique values:")
print(df['Floor'].unique())
print("\n")

print("New 'current_floor' unique values:")
print(df['current_floor'].unique())
print("\n")

print("New 'total_floors' unique values:")
print(df['total_floors'].unique())
print("\n")

display(df[['Floor', 'current_floor', 'total_floors']].head())



Numerical values from floors

In [ ]:
df = df[df["Area Type"] != "Built Area"].copy()

min_n = 20
counts = df["Area Locality"].value_counts()
keep = counts[counts >= min_n].index

df["AreaLocality_grp"] = np.where(df["Area Locality"].isin(keep),
                                  df["Area Locality"],
                                  "Other")

print(f"Unique values for AreaLocality_grp:")
print(df["AreaLocality_grp"].unique())
print("\n")
print(f"Column 'Arealocality_grp': {df["AreaLocality_grp"].nunique()} unique values")



Creating "popular localities that can giv esome info" and "other"

In [ ]:
sns.histplot(df["Rent"], bins=60)
plt.yscale("log")
plt.title("Rent (log y-scale)")
plt.show()

sns.histplot(df["log_rent"], bins=60)
plt.title("log(Rent)")
plt.show()

sns.scatterplot(data=df, x="Size", y="log_rent", hue="City", alpha=0.4, linewidth=0)
plt.title("Rent vs Size (log y-scale)")
plt.show()

sns.boxplot(data=df, x="City", y="log_rent")
plt.xticks(rotation=30, ha="right")
plt.title("log_rent by City")
plt.show()


In [ ]:
formula_loc_grp = """
log_rent ~ Size + BHK + Bathroom
+ C(City) + C(Q("Furnishing Status")) + C(Q("Tenant Preferred")) + C(Q("Area Type"))
+ C(AreaLocality_grp)
"""
m_loc_grp = smf.ols(formula_loc_grp, data=df).fit(cov_type="HC3")
print(m_loc_grp.summary())

formula_area_interact = """
log_rent ~ Size:C(Q("City")) + BHK + Bathroom
+ C(City) + C(Q("Furnishing Status")) + C(Q("Tenant Preferred"))
+ C(AreaLocality_grp)
"""
m_area = smf.ols(formula_area_interact, data=df).fit(cov_type="HC3")
print(m_area.summary())

formula_area_interact_floors = """
log_rent ~ Size:C(Q("City")) + BHK + Bathroom current_floor*total_floors
+ C(City) + C(Q("Furnishing Status")) + C(Q("Tenant Preferred"))
+ C(AreaLocality_grp)
"""
m_floors = smf.ols(formula_area_interact_floors, data=df).fit(cov_type="HC3")
print(m_floors.summary())

formula_area_interact_floors2 = """
log_rent ~ Size:C(Q("City")) + BHK + Bathroom + current_floor+total_floors
+ C(City) + C(Q("Furnishing Status")) + C(Q("Tenant Preferred"))
+ C(AreaLocality_grp)
"""
m_floors2 = smf.ols(formula_area_interact_floors2, data=df).fit(cov_type="HC3")
print(m_floors2.summary())

model floors seems the best to me, because then there is not such a problem with the colinearity with total floors and  current floor and yet seems to have nicce power - not the final model, just testing for now

In [ ]:

def loocv_rmse_mae_ols(formula, data, y_col="log_rent", cov_type=None):
    fit = smf.ols(formula, data=data).fit() if cov_type is None else smf.ols(formula, data=data).fit(cov_type=cov_type)

    infl = OLSInfluence(fit)
    h = infl.hat_matrix_diag
    e = fit.resid.values
    press = e / (1.0 - h)

    rmse = np.sqrt(np.mean(press**2))
    mae  = np.mean(np.abs(press))
    return rmse, mae, fit

In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence

formulas = {
    "Model1": formula_loc_grp,
    "Model2": formula_area_interact,
    "Model3": formula_area_interact_floors,
    "Model4": formula_area_interact_floors2,
}

results = {}
for name, f in formulas.items():
    rmse, mae, _ = loocv_rmse_mae_ols(f, df, cov_type="HC3")
    results[name] = {"PRESS_RMSE_log": rmse, "PRESS_MAE_log": mae}

pd.DataFrame(results).T.sort_values("PRESS_RMSE_log")

Model 3 - the one wit hthe floor interaction - looks good here also - I start with this maximal model and then move down

In [ ]:
formula_best = formula_area_interact_floors
m_best = m_floors

In [ ]:
df_diag = df.copy()
df_diag["y"] = df_diag["log_rent"]
df_diag["yhat"] = m_best.fittedvalues
df_diag["resid"] = m_best.resid

influence = m_best.get_influence()

df_diag["resid_studentized"] = influence.resid_studentized
df_diag["hat"] = influence.hat_matrix_diag
df_diag["cooks_d"] = influence.cooks_distance[0]

from statsmodels.graphics.gofplots import qqplot
sns.scatterplot(data=df_diag, x="yhat", y="resid", alpha=0.35, linewidth=0)
plt.axhline(0, linewidth=1)
plt.title("Residuals vs Fitted (log space)")
plt.show()

qqplot(df_diag["resid_studentized"], line="45")
plt.title("QQ plot of residuals")
plt.show()

sns.histplot(df_diag["resid"], bins=60)
plt.title("Residual distribution (log space)")
plt.show()

sns.histplot(df_diag["resid_studentized"], bins=60)
plt.title("Histogram of Studentized Residuals")
plt.show()

In [ ]:
import statsmodels.api as sm

sns.scatterplot(data=df_diag, x="hat", y=np.abs(df_diag["resid_studentized"]),
                size="cooks_d", sizes=(20, 400), alpha=0.6, linewidth=0, hue='cooks_d', palette='viridis')
plt.axhline(2, linestyle="--", color='grey', label="|Studentized Residual| = 2")
plt.axhline(3, linestyle="--", color='red', label="|Studentized Residual| = 3")
plt.title("Influence Plot: Leverage vs. Absolute Studentized Residuals (size ~ Cook's D)")
plt.xlabel("Leverage (Hat value)")
plt.ylabel("|Studentized Residuals|")
plt.legend(title="Legend")
plt.tight_layout()
plt.show()


sns.scatterplot(data=df_diag, x="hat", y=np.abs(df_diag["resid_studentized"]),
                size="cooks_d", sizes=(20, 400), alpha=0.6, linewidth=0, hue='cooks_d', palette='viridis')
plt.axhline(2, linestyle="--", color='grey', label="|Studentized Residual| = 2")
plt.axhline(3, linestyle="--", color='red', label="|Studentized Residual| = 3")
plt.title("Influence Plot: Leverage vs. Absolute Studentized Residuals (size ~ Cook's D)")
plt.xlabel("Leverage (Hat value)")
plt.ylabel("|Studentized Residuals|")
plt.xscale('log') # Leverage can be skewed, log scale might help visualization
plt.legend(title="Legend")
plt.tight_layout()
plt.show()


In [ ]:
influence = m_best.get_influence()

df_diag["hat"] = influence.hat_matrix_diag
df_diag["stud"] = influence.resid_studentized_external
df_diag["cooks_d"] = influence.cooks_distance[0]

# Top influential
top_cook = df_diag.sort_values("cooks_d", ascending=False).head(15)
top_out  = df_diag.reindex(df_diag["stud"].abs().sort_values(ascending=False).index).head(15)

cols = ["Rent","log_rent","yhat","resid","stud","hat","cooks_d"]

top_cook[cols], top_out[cols]

In [ ]:
top_idx = [1837, 3656, 4076]

tmp = df_diag.loc[top_idx, ["Rent","log_rent","yhat","resid","stud","hat","cooks_d"]].copy()
tmp["ratio_actual_to_pred"] = np.exp(tmp["resid"])          # ~ actual/predicted
tmp["abs_pct_error_like"] = np.abs(tmp["ratio_actual_to_pred"] - 1)

tmp.sort_values("cooks_d", ascending=False)




In [ ]:
df.loc[top_idx]

In [ ]:
dfbetas = pd.DataFrame(influence.dfbetas, columns=m_best.params.index, index=df.index)

# which observations most affect coefficients (max abs dfbeta across params)
df_diag["max_abs_dfbeta"] = dfbetas.abs().max(axis=1)

df_diag.sort_values("max_abs_dfbeta", ascending=False).head(15)[
    ["Rent","Size","BHK","Bathroom","stud","hat","cooks_d","max_abs_dfbeta"]
]


In [ ]:
suspects = [4185, 2656, 4696, 3019, 4457]
df.loc[suspects]

df["Area Type"].value_counts(dropna=False)


In [ ]:
#Nejpravděpoodobněji se jedná o chyba týden místo měsíc / rok místo měsíc - proto upravit
df_old = df.copy()
df = df_old.copy()

df.loc[1837, 'Rent'] = df.loc[1837, 'Rent']/np.float64(12)
df.loc[3656, 'Rent'] = df.loc[3656, 'Rent']/np.float64(12)
df.loc[4076, 'Rent'] = df.loc[3656, 'Rent']*np.float64(4.3)
bad_clear = [2656, 4185]


df = df[df["Area Type"] != "Built Area"].copy()

df = df.drop(index=bad_clear, errors="ignore")
df["log_rent"] = np.log(df["Rent"])

#A tyhle mi přišly jako chybné - proto vyhodit
m_drop = smf.ols(m_best.model.formula, data=df).fit(cov_type="HC3")
print(m_drop.summary())

m_best = m_drop

In [ ]:
infl = m_best.get_influence()
df_diag = df.copy()
df_diag["y"] = df_diag["log_rent"]
df_diag["yhat"] = m_best.fittedvalues
df_diag["resid"] = m_best.resid

df_diag["hat"] = infl.hat_matrix_diag
df_diag["stud"] = infl.resid_studentized_external
df_diag["cooks_d"] = infl.cooks_distance[0]

# Top influential
top_cook = df_diag.sort_values("cooks_d", ascending=False).head(15)
top_out  = df_diag.reindex(df_diag["stud"].abs().sort_values(ascending=False).index).head(15)

cols = ["Rent","log_rent","yhat","resid","stud","hat","cooks_d"]

top_cook[cols], top_out[cols]


In [ ]:
import patsy

y, X = patsy.dmatrices(formula_area_interact_floors, data=df, return_type="dataframe")
X.shape, y.shape

svals = np.linalg.svd(X.values, compute_uv=False)
kappa = svals[0] / svals[-1]
kappa


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame({
    "term": X.columns,
    "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
}).sort_values("VIF", ascending=False)

vif.head(30)


In [ ]:
import patsy

y, X = patsy.dmatrices(formula_best, data=df, return_type="dataframe")

# drop intercept for correlation
X2 = X.drop(columns=["Intercept"], errors="ignore")

top_cols = X2.var().sort_values(ascending=False).head(30).index
corrX = X2[top_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corrX, center=0, square=True)
plt.title("Correlation matrix (design matrix subset)")
plt.show()


In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt


terms = ["Size", "current_floor", "total_floors", "BHK", "Bathroom",]

for t in terms:
    cols = [i for i, name in enumerate(m_best.model.exog_names) if name == t]
    if not cols:
        continue
    sm.graphics.plot_ccpr(m_best, cols[0])
    plt.title(f"CCPR (partial residual) for {t}")
    plt.show()


In [ ]:

exog_names = list(m_best.model.exog_names)

def plot_ccpr_by_name(model, name):
    exog = list(model.model.exog_names)
    if name not in exog:
        print(f"Not found: {name}")
        return
    idx = exog.index(name)
    sm.graphics.plot_ccpr(model, idx)
    plt.title(f"CCPR (partial residual) for {name}")
    plt.show()

# --- 1) floor interaction (either order) ---
floor_terms = [n for n in exog_names if ("current_floor" in n and "total_floors" in n and ":" in n)]
print("Floor interaction terms found:", floor_terms)
for t in floor_terms:
    plot_ccpr_by_name(m_best, t)

# --- 2) Size × Area Type interaction terms (use your actual naming) ---
area_terms = [n for n in exog_names if n.startswith('Size:C(Q("Area Type"))[')]
print("Area Type interaction terms found:", area_terms)
for t in area_terms:
    plot_ccpr_by_name(m_best, t)


These look Roughly linear but not 100%

In [ ]:
import numpy as np

df2 = df.copy()
df2["log_Size"] = np.log(df2["Size"])
df2["sqrt_Size"] = np.sqrt(df2["Size"])



In [ ]:
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd

def loocv_press(formula, data):
    m = smf.ols(formula, data=data).fit()
    infl = m.get_influence()
    h = infl.hat_matrix_diag
    e = m.resid.values
    loo_resid = e / (1 - h)
    rmse = float(np.sqrt(np.mean(loo_resid**2)))
    mae = float(np.mean(np.abs(loo_resid)))
    return rmse, mae, m


In [ ]:
from patsy import bs

base_formula = m_best.model.formula

formulas = {
    "baseline": base_formula,

    "log(Size)": base_formula.replace("Size", "log_Size"),
    "sqrt(Size)": base_formula.replace("Size", "sqrt_Size"),

    "bs(Size,df=5)": base_formula.replace("Size", "bs(Size, df=5, degree=3)")
    }


rows = []
models = {}
for name, f in formulas.items():
    rmse, mae, m = loocv_press(f, df2)
    rows.append((name, rmse, mae, m.aic, m.bic))
    models[name] = m

res = pd.DataFrame(rows, columns=["model","LOO_RMSE_log","LOO_MAE_log","AIC","BIC"]).sort_values("LOO_RMSE_log")
res


In [ ]:
df_diag = df.copy()
df_diag["y"] = df_diag["log_rent"]
df_diag["yhat"] = m_best.fittedvalues
df_diag["resid"] = m_best.resid

influence = m_best.get_influence()

df_diag["resid_studentized"] = influence.resid_studentized
df_diag["hat"] = influence.hat_matrix_diag
df_diag["cooks_d"] = influence.cooks_distance[0]

sns.scatterplot(data=df_diag, x="yhat", y="resid", alpha=0.35, linewidth=0)
plt.axhline(0, linewidth=1)
plt.title("Residuals vs Fitted (log space)")
plt.show()

qqplot(df_diag["resid_studentized"], line="45")
plt.title("QQ plot of residuals")
plt.show()

sns.histplot(df_diag["resid"], bins=60)
plt.title("Residual distribution (log space)")
plt.show()

sns.histplot(df_diag["resid_studentized"], bins=60)
plt.title("Histogram of Studentized Residuals")
plt.show()



A few too heavy on the right tail - makes sense, probably some extra luxurious location not accounted for

In [ ]:


plt.figure(figsize=(10, 8))
sns.scatterplot(x=m_best.fittedvalues, y=m_best.resid, alpha=0.35, linewidth=0)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted')
plt.show()


Question answers:

Adding more predictors

Training error: always goes down (or stays the same).

Test/CV error: often goes down at first, then flattens, and can go back up (overfitting).

Adding interaction terms

Helps only if the interaction is real (e.g., Size effect differs by City).

Otherwise it increases variance a lot → test error often worsens unless regularized.

#CLEANING HAS BEEN DONE TO MAKE THE MAXIMAL MODEL USABLE

NOW ONTO THE OTHER ONES - Train split and all

The split is done so that cities are balanced

In [ ]:
from sklearn.model_selection import train_test_split
df_use = df.copy()

train_df, val_df = train_test_split(
    df_use,
    test_size=0.2,
    random_state=0,
    stratify=df_use["City"]
)


train_df["floor_frac"] = train_df["current_floor"] / train_df["total_floors"].replace(0, np.nan)
val_df["floor_frac"]   = val_df["current_floor"]   / val_df["total_floors"].replace(0, np.nan)

train_df["floor_frac"] = train_df["floor_frac"].fillna(0)
val_df["floor_frac"]   = val_df["floor_frac"].fillna(0)


train_df.shape, val_df.shape

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

for col in train_df.columns:
    print(f"\n=== {col} ===")

    # numeric columns: overlay histograms (density)
    if pd.api.types.is_numeric_dtype(train_df[col]):
        plt.figure()
        sns.histplot(train_df[col], stat="density", bins=30, alpha=0.35, label="train")
        sns.histplot(val_df[col],   stat="density", bins=30, alpha=0.35, label="val")
        plt.title(f"{col} (numeric)")
        plt.legend()
        plt.show()

    # everything else: show normalized value counts side-by-side
    else:
        vc_train = train_df[col].value_counts(normalize=True)
        vc_val   = val_df[col].value_counts(normalize=True)

        dist = pd.concat([vc_train.rename("train"), vc_val.rename("val")], axis=1).fillna(0)
        print(dist.sort_values("train", ascending=False))

        # plot as bar chart
        dist.plot(kind="bar", figsize=(10, 4))
        plt.title(f"{col} (categorical) – normalized frequencies")
        plt.ylabel("proportion")
        plt.tight_layout()
        plt.show()


Maybe the floors could introduce a bit of a problem

In [ ]:
def score_model(formula, train_df, val_df):
    m = smf.ols(formula, data=train_df).fit(cov_type="HC3")

    pred_log = m.predict(val_df)
    y_log = val_df["log_rent"].values
    err_log = y_log - pred_log.values
    rmse_log = float(np.sqrt(np.mean(err_log**2)))
    mae_log  = float(np.mean(np.abs(err_log)))

    pred_rent = np.exp(pred_log)
    y_rent = val_df["Rent"].values
    err_rent = y_rent - pred_rent.values
    rmse_rent = float(np.sqrt(np.mean(err_rent**2)))
    mae_rent  = float(np.mean(np.abs(err_rent)))

    return m, rmse_log, mae_log, rmse_rent, mae_rent

baseline_formula = """
log_rent ~ Size * C(Q("Area Type")) + BHK + Bathroom + current_floor*total_floors
+ C(City) + C(Q("Furnishing Status")) + C(Q("Tenant Preferred"))
+ C(AreaLocality_grp) + C(Q("Area Type"))
"""

m0, rmse_log, mae_log, rmse_rent, mae_rent = score_model(baseline_formula, train_df, val_df)

results0 = pd.DataFrame([[
    "baseline", rmse_log, mae_log, rmse_rent, mae_rent, m0.aic, m0.bic
]], columns=["model","RMSE_log","MAE_log","RMSE_Rent","MAE_Rent","AIC_train","BIC_train"])

results0


In [ ]:
def fit_bic(formula, data):
    m = smf.ols(formula, data=data).fit()
    return m, float(m.bic), float(m.aic)

def build_formula(fixed_terms, optional_terms):
    rhs = fixed_terms + optional_terms
    rhs = [t.strip() for t in rhs if t.strip()]
    return "log_rent ~ " + " + ".join(rhs)

def stepwise_bic(train_df, fixed_terms, start_optional, optional_pool, max_steps=50, verbose=True):
    current_opt = list(start_optional)
    current_formula = build_formula(fixed_terms, current_opt)
    current_model, current_bic, current_aic = fit_bic(current_formula, train_df)

    if verbose:
        print(f"Start BIC: {current_bic:.3f}")
        print(current_formula)

    for step in range(1, max_steps + 1):
        candidates = []
        curset = set(current_opt)

        # add-one
        for t in optional_pool:
            if t in curset:
                continue
            new_opt = list(curset | {t})
            f = build_formula(fixed_terms, new_opt)
            try:
                m, bic, aic = fit_bic(f, train_df)
                candidates.append(("add", t, new_opt, f, m, bic, aic))
            except Exception:
                pass

        # remove-one (only from current optional set)
        for t in list(curset):
            newset = set(curset)
            newset.remove(t)
            new_opt = list(newset)
            f = build_formula(fixed_terms, new_opt)
            try:
                m, bic, aic = fit_bic(f, train_df)
                candidates.append(("remove", t, new_opt, f, m, bic, aic))
            except Exception:
                pass

        if not candidates:
            break

        best = min(candidates, key=lambda x: x[5])
        move, term, best_opt, best_formula, best_model, best_bic, best_aic = best

        if best_bic + 1e-9 < current_bic:
            current_opt = best_opt
            current_formula = best_formula
            current_model = best_model
            current_bic = best_bic
            current_aic = best_aic
            if verbose:
                print(f"\nStep {step}: {move} {term}")
                print(f"  BIC -> {current_bic:.3f} | AIC -> {current_aic:.3f}")
        else:
            if verbose:
                print("\nNo further BIC improvement. Stop.")
            break

    return current_model, current_formula, current_bic, current_aic


In [ ]:
fixed_terms = ["C(City)"]

# Start from the baseline (minus City, which is fixed)
start_optional = [
    "Size", "BHK", "Bathroom", "floor_frac",
    "current_floor", "total_floors", "current_floor:total_floors",
    'C(Q("Furnishing Status"))',
    'C(Q("Tenant Preferred"))',
    "C(AreaLocality_grp)",
    'C(Q("Area Type"))', "Size:C(AreaLocality_grp)"
]

optional_pool = start_optional.copy() + ["Size:C(City)"]

best_train_model, best_formula, best_bic, best_aic = stepwise_bic(
    train_df=train_df,
    fixed_terms=fixed_terms,
    start_optional=start_optional,
    optional_pool=optional_pool,
    max_steps=50,
    verbose=True
)

print("\n=== Best by TRAIN BIC ===")
print("BIC:", best_bic)
print("AIC:", best_aic)
print(best_formula)

In [ ]:
#WHAT ABOUT IF WITH FLOOR FRAC and from floor up?

fixed_terms = ["C(City)"]

optional_pool = [
    "Size", "BHK", "Bathroom",
    "current_floor", "total_floors", "current_floor:total_floors",
    "floor_frac",
    'C(Q("Furnishing Status"))',
    'C(Q("Tenant Preferred"))',
    "C(AreaLocality_grp)",
    'C(Q("Area Type"))', "Size:C(City)", "Size:C(AreaLocality_grp)"
]

start_optional = ["Size"]

best_train_model, best_formula, best_bic, best_aic = stepwise_bic(
    train_df=train_df,
    fixed_terms=fixed_terms,
    start_optional=start_optional,
    optional_pool=optional_pool,
    max_steps=50,
    verbose=True
)

print("\n=== Best by TRAIN BIC ===")
print("BIC:", best_bic)
print("AIC:", best_aic)
print(best_formula)


THE SAME RESULT, SO THIS IS THE DEFINITE PREFFERED MODEL.

I focus on "Interpretability" and not the best predictive power - that is why I focus on BIC/AIC. If we wanted the highest predictiv epower on this type of data, it could start to get ugly real fast.

In [ ]:

formula_city_slope = """
log_rent ~ C(City) + C(Q("Tenant Preferred")) + Bathroom
+ C(AreaLocality_grp) + C(Q("Furnishing Status")) + BHK + C(Q("Area Type"))
+ Size * C(City) + total_floors
"""


f_no_int = """
log_rent ~ C(City) + C(Q("Tenant Preferred")) + Bathroom
+ C(AreaLocality_grp) + C(Q("Furnishing Status")) + BHK + C(Q("Area Type"))
+ Size + total_floors
"""

f_int = formula_city_slope

m_no = smf.ols(f_no_int, data=train_df).fit()
m_in = smf.ols(f_int,   data=train_df).fit()

_, rmse0, mae0, _, _ = score_model(f_no_int, train_df, val_df)
_, rmse1, mae1, _, _ = score_model(f_int,    train_df, val_df)

rmse0, rmse1, mae0, mae1


In [ ]:
import pandas as pd

params = m_in.params

base_city = [c for c in train_df["City"].unique() if f"C(City)[T.{c}]" not in params.index][0]
base_slope = params["Size"]

slopes = []
for city in sorted(train_df["City"].unique()):
    adj_name = f"Size:C(City)[T.{city}]"
    slope = base_slope + (params[adj_name] if adj_name in params.index else 0.0)
    slopes.append((city, slope))

pd.DataFrame(slopes, columns=["City", "d(log_rent)/d(Size)"]).sort_values("d(log_rent)/d(Size)", ascending=False)


Seems that the interaction really makes sense.

Now to check for overfit:

In [ ]:
def fit_and_error(formula, train_df, val_df):
    m = smf.ols(formula, data=train_df).fit()

    pred_tr = m.predict(train_df)
    pred_va = m.predict(val_df)

    err_tr = train_df["log_rent"].values - pred_tr.values
    err_va = val_df["log_rent"].values   - pred_va.values

    out = pd.DataFrame([{
        "RMSE_train": float(np.sqrt(np.mean(err_tr**2))),
        "RMSE_val":   float(np.sqrt(np.mean(err_va**2))),
        "MAE_train":  float(np.mean(np.abs(err_tr))),
        "MAE_val":    float(np.mean(np.abs(err_va))),
        "RMSE_gap":   float(np.sqrt(np.mean(err_va**2)) - np.sqrt(np.mean(err_tr**2))),
        "MAE_gap":    float(np.mean(np.abs(err_va)) - np.mean(np.abs(err_tr))),
        "R2_train":   float(m.rsquared),
        "n_train":    int(len(train_df)),
        "n_val":      int(len(val_df)),
    }])
    return m, out

m, err_table = fit_and_error(formula_city_slope, train_df, val_df)
err_table